In [12]:
import numpy as np
from itertools import product
from scipy.optimize import fsolve


class model(object):
    """
    model
    Attributes
    ----------
    n : int
        number of players
    alpha : float
        product differentiation parameter
    beta : float
        exploration parameter
    delta : float
        discount factor
    mu : float
        product differentiation parameter
    a : int
        value of the products
    a0 : float
        value of the outside option
    c : float
        marginal cost
    k : int
        dimension of the grid
    stable: int
        periods of game stability
    """

    def __init__(self, **kwargs):
        """Initialize game with default values"""
        # Default properties
        self.n = kwargs.get('n', 2)
        self.alpha = kwargs.get('alpha', 0.15)
        self.beta = kwargs.get('beta', 4e-6)
        self.delta = kwargs.get('delta', 0.95)
        self.c = kwargs.get('c', 1)
        self.a = kwargs.get('a', 2)
        self.a0 = kwargs.get('a0', 0)
        self.mu = kwargs.get('mu', 0.25)
        self.k = kwargs.get('k', 15)
        self.tstable = kwargs.get('tstable', 1e5)
        self.tmax = kwargs.get('tstable', 1e7)

        # Derived properties
        self.sdim, self.s0 = self.init_state()
        self.p_minmax = self.compute_p_competitive_monopoly()
        self.A = self.init_actions()
        self.PI = self.init_PI()
        self.Q = self.init_Q()

    def demand(self, p):
        """Computes demand"""
        e = np.exp((self.a - p) / self.mu)
        d = e / (np.sum(e) + np.exp(self.a0 / self.mu))
        return d

    def foc(self, p):
        """Compute first order condition"""
        d = self.demand(p)
        zero = 1 - (p - self.c) * (1 - d) / self.mu
        return np.squeeze(zero)

    def foc_monopoly(self, p):
        """Compute first order condition of a monopolist"""
        d = self.demand(p)
        d1 = np.flip(d)
        p1 = np.flip(p)
        zero = 1 - (p - self.c) * (1 - d) / self.mu + (p1 - self.c) * d1 / self.mu
        return np.squeeze(zero)

    def compute_p_competitive_monopoly(self):
        """Computes competitive and monopoly prices"""
        p0 = np.ones((1, self.n)) * 3 * self.c
        p_competitive = fsolve(self.foc, p0)
        p_monopoly = fsolve(self.foc_monopoly, p0)
        return p_competitive, p_monopoly

    def init_actions(self):
        """Get action space of the firms"""
        a = np.linspace(min(self.p_minmax[0]), max(self.p_minmax[1]), self.k - 2)
        delta = a[1] - a[0]
        A = np.linspace(min(a) - delta, max(a) + delta, self.k)
        return A

    def init_state(self):
        """Get state dimension and initial state"""
        sdim = (self.k, self.k)
        s0 = np.zeros(len(sdim)).astype(int)
        return sdim, s0

    def compute_profits(self, p):
        """Compute payoffs"""
        d = self.demand(p)
        pi = (p - self.c) * d
        return pi

    def init_PI(game):
        """Initialize Profits (k^n x kp x n)"""
        PI = np.zeros(game.sdim + (game.n,))
        for s in product(*[range(i) for i in game.sdim]):
            p = np.asarray(game.A[np.asarray(s)])
            print(p)
            PI[s] = game.compute_profits(p)
        return PI

    def init_Q(game):
        """Initialize Q function (n x #s x k)"""
        Q = np.zeros((game.n,) + game.sdim + (game.k,))
        for n in range(game.n):
            pi = np.mean(game.PI[:, :, n], axis=1 - n)
            Q[n] = np.tile(pi, game.sdim + (1,)) / (1 - game.delta)
        return Q

In [14]:
sr = model()
sr.sdim

[1.43525547 1.43525547]
[1.43525547 1.47292666]
[1.43525547 1.51059785]
[1.43525547 1.54826904]
[1.43525547 1.58594022]
[1.43525547 1.62361141]
[1.43525547 1.6612826 ]
[1.43525547 1.69895379]
[1.43525547 1.73662498]
[1.43525547 1.77429617]
[1.43525547 1.81196735]
[1.43525547 1.84963854]
[1.43525547 1.88730973]
[1.43525547 1.92498092]
[1.43525547 1.96265211]
[1.47292666 1.43525547]
[1.47292666 1.47292666]
[1.47292666 1.51059785]
[1.47292666 1.54826904]
[1.47292666 1.58594022]
[1.47292666 1.62361141]
[1.47292666 1.6612826 ]
[1.47292666 1.69895379]
[1.47292666 1.73662498]
[1.47292666 1.77429617]
[1.47292666 1.81196735]
[1.47292666 1.84963854]
[1.47292666 1.88730973]
[1.47292666 1.92498092]
[1.47292666 1.96265211]
[1.51059785 1.43525547]
[1.51059785 1.47292666]
[1.51059785 1.51059785]
[1.51059785 1.54826904]
[1.51059785 1.58594022]
[1.51059785 1.62361141]
[1.51059785 1.6612826 ]
[1.51059785 1.69895379]
[1.51059785 1.73662498]
[1.51059785 1.77429617]
[1.51059785 1.81196735]
[1.51059785 1.84

(15, 15)

In [4]:
"""
Q-learning Functions
"""

import sys
import numpy as np


def pick_strategies(game, s, t):
    """Pick strategies by exploration vs exploitation"""
    a = np.zeros(game.n).astype(int)
    pr_explore = np.exp(- t * game.beta)
    e = (pr_explore > np.random.rand(game.n))
    for n in range(game.n):
        if e[n]:
            a[n] = np.random.randint(0, game.k)
        else:
            a[n] = np.argmax(game.Q[(n,) + tuple(s)])
    return a


def update_q(game, s, a, s1, pi, stable):
    """Update Q matrix"""
    for n in range(game.n):
        subj_state = (n,) + tuple(s) + (a[n],)
        old_value = game.Q[subj_state]
        max_q1 = np.max(game.Q[(n,) + tuple(s1)])
        new_value = pi[n] + game.delta * max_q1
        old_argmax = np.argmax(game.Q[(n,) + tuple(s)])
        game.Q[subj_state] = (1 - game.alpha) * old_value + game.alpha * new_value
        # Check stability
        new_argmax = np.argmax(game.Q[(n,) + tuple(s)])
        same_argmax = (old_argmax == new_argmax)
        stable = (stable + same_argmax) * same_argmax
    return game.Q, stable


def check_convergence(game, t, stable):
    """Check if game converged"""
    if (t % game.tstable == 0) & (t > 0):
        sys.stdout.write("\rt=%i" % t)
        sys.stdout.flush()
    if stable > game.tstable:
        print('Converged!')
        return True
    if t == game.tmax:
        print('ERROR! Not Converged!')
        return True
    return False


def simulate_game(game):
    """Simulate game"""
    s = game.s0
    stable = 0
    # Iterate until convergence
    for t in range(int(game.tmax)):
        a = pick_strategies(game, s, t)
        pi = game.PI[tuple(a)]
        s1 = a
        game.Q, stable = update_q(game, s, a, s1, pi, stable)
        s = s1
        if check_convergence(game, t, stable):
            print(s1)
            break
    return game

In [5]:


# Init algorithm
game = model()

# Compute equilibrium
game_equilibrium = simulate_game(game)

t=1700000Converged!
[14 13]


In [1]:
import numpy as np

In [2]:
game.

1

In [5]:
game.init_Q()

array([[[[5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         ...,
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608]],

        [[5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.16802608],
         ...,
         [5.84509262, 6.04624758, 6.18619942, ..., 4.78307397,
          4.47805956, 4.1

In [6]:
game.init_PI()

array([[[0.20682553, 0.20682553],
        [0.22155186, 0.2070539 ],
        [0.23600529, 0.20482047],
        [0.2500352 , 0.20041303],
        [0.26350889, 0.19415033],
        [0.27631598, 0.18636658],
        [0.28837091, 0.17739646],
        [0.29961382, 0.16756186],
        [0.31000971, 0.15716109],
        [0.31954627, 0.14646121],
        [0.32823096, 0.13569327],
        [0.3360875 , 0.12505043],
        [0.34315225, 0.11468832],
        [0.34947076, 0.10472711],
        [0.35509455, 0.0952549 ]],

       [[0.2070539 , 0.22155186],
        [0.22292666, 0.22292666],
        [0.23866335, 0.22163041],
        [0.25409097, 0.21792449],
        [0.26905004, 0.2121133 ],
        [0.2834008 , 0.2045291 ],
        [0.29702771, 0.19551608],
        [0.30984202, 0.18541533],
        [0.32178241, 0.17455169],
        [0.33281401, 0.16322342],
        [0.34292597, 0.15169495],
        [0.35212819, 0.14019278],
        [0.36044761, 0.12890402],
        [0.36792429, 0.11797738],
        [0.3

In [11]:
sr = model()
sr.PI.shape

(15, 15, 2)